In [25]:
import numpy as np
import os
import cv2
import keras
from keras.models import load_model

# Load the saved model
model = load_model('../models/adv_lstm_deepfake_image.h5')

# Reuse your existing functions
IMG_SIZE = 256
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [26]:

# Define the feature extractor (same as before)
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

feature_extractor = build_feature_extractor()

# Function to preprocess video
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]  # Convert from BGR to RGB
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

def prepare_video_for_prediction(video_path):
    # Load and preprocess video frames
    frames = load_video(video_path)
    
    # Extract features and prepare masks
    frame_masks = np.zeros((1, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros((1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # Extract features from each frame individually
    video_length = frames.shape[0]  # Get the number of frames
    length = min(MAX_SEQ_LENGTH, video_length)
    
    for j in range(length):
        # Pass one frame at a time to the feature extractor
        frame = frames[j]  # Select the j-th frame
        frame = np.expand_dims(frame, axis=0)  # Add batch dimension (1, 256, 256, 3)
        
        frame_features[0, j, :] = feature_extractor.predict(frame)
        
    frame_masks[0, :length] = 1  # Mark the frames as not masked

    return frame_features, frame_masks

# Use the model for prediction
def predict_on_custom_video(video_path):
    frame_features, frame_masks = prepare_video_for_prediction(video_path)
    prediction = model.predict([frame_features, frame_masks])
    return prediction

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


# Example usage:
video_path = r"\C:\Users\shafe\Downloads\vid\vid.mp4"
prediction = predict_on_custom_video(video_path)
print(f"Prediction (FAKE=1, REAL=0): {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 889ms/step
Prediction (FAKE=1, REAL=0): [[0.5244092]]


In [27]:
import numpy as np

# Function to use the model for prediction and print frame-wise predictions
def predict_on_custom_video(video_path):
    frame_features, frame_masks = prepare_video_for_prediction(video_path)
    predictions = []

    # Iterate through each frame's features
    for i in range(frame_features.shape[1]):  # Loop through the MAX_SEQ_LENGTH
        frame_prediction = model.predict([frame_features[:, i:i+1, :], frame_masks[:, i:i+1]])
        predictions.append(frame_prediction[0][0])  # Store the scalar prediction for the frame

    # Calculate average prediction
    average_prediction = np.mean(predictions)
    return predictions, average_prediction

# Example usage:
video_path = r"C:\Users\shafe\Downloads\test_videos\zzmgnglanj.mp4"  # Fixed backslash here
frame_predictions, average_prediction = predict_on_custom_video(video_path)

# Print predictions for each frame
for idx, pred in enumerate(frame_predictions):
    print(f"Frame {idx + 1} Prediction (FAKE=1, REAL=0): {pred:.4f}")

# Print the average prediction
print(f"\nAverage Prediction (FAKE=1, REAL=0): {average_prediction:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━

In [28]:
if(prediction>=0.5):
    print(f'The video is FAKE')
else:
    print(f'The video is REAL')

The video is FAKE
